In [29]:
%reload_ext autoreload
%autoreload 2
import tensorflow as tf
import sys, os, random, copy, collections, time, re, argparse
sys.stdour = sys.stderr
import numpy as np
from pprint import pprint
from utils import common
from core import models, datasets
from core.vocabularies import WordVocabularyWithEmbedding

from utils.common import dotDict
from main import Manager
sys.stdout = sys.stderr
args = dotDict({
    'checkpoint_path': 'checkpoints/tmp',
    'mode': 'train',
    'config_path': 'configs/config',
    'cleanup': True,
    'log_file': None,
    'interactive': True,
})

graph = tf.Graph()
sess = tf.InteractiveSession(graph=graph)
sess.as_default()
manager = Manager(args, sess)
config = manager.config

In [30]:
vocab = WordVocabularyWithEmbedding(config.embeddings, vocab_size=config.vocab_size, lowercase=config.lowercase)

Loading word embeddings from dataset/embeddings/glove.840B.300d.txt.filtered...
Done loading word embeddings.


In [48]:
config.batch_size=2
dataset_type = getattr(datasets, config.dataset_type)
dataset = dataset_type(config.dataset_path, vocab)
print dataset
sys.stdout=sys.stderr
print dataset.tensorized
for s,t in zip(*dataset.tensorized):
    print s, t

([['$', '000.00', 'to', '$', '0,000.00', 'upfront', '...', 'then', '$', '000.00', ',', '$', '000.00', ',', 'or', '$', '000.00', 'per', 'month', 'after', 'that', '?'], ['the', 'fee', 'for', 'taking', 'part', 'in', 'the', 'pella', 'project', 'is', 'estimated', 'to', 'be', 'around', '$', '0000', '(', 'australian', 'dollars', ')', 'p'], ['the', 'last', 'time', 'i', 'checked', ',', 'i', 'was', 'told', 'that', 'it', 'would', 'cost', 'the', 'astronomical', 'figure'], ['of', '$', '000,000', 'to', 'raise', '$', '0.0', 'trillion'], ['the', 'cost', 'of', 'the', 'program', 'is', 'generally', 'either', '$', '00', 'or', '$', '000', 'per', 'month'], ['a', 'smoke', 'alarm', 'may', 'be', 'purchased', 'at', 'most', 'retail', 'stores', 'for', 'about', '$', '00.00', '-', '$', '00.00', '.'], ['a.', 'lumineerscerinateare', 'community', 'users', 'submit', 'preserving', '000.00', 'the', 'average', 'being', 'per', 'w', '/', 'an', '$', '000', '$', '0'], ['around', '$', '000', 'per', 'mbps', 'per', 'month'], ['a

In [44]:
with tf.variable_scope('', reuse=tf.AUTO_REUSE):
    model = manager.create_model(manager.config, vocab)
    #dataset = dataset_type(config.dataset_path, self.vocab)
    

attention_states Tensor("Encoder_5/Encoder/transpose:0", shape=(?, 30, 50), dtype=float32)
encoder_inputs Tensor("embedding_lookup_5:0", shape=(?, 30, 300), dtype=float32)
decoder_inputs Tensor("Placeholder_5/concat:0", shape=(?, 6), dtype=int32)
attention_states Tensor("Encoder_5/Encoder/transpose:0", shape=(?, 30, 50), dtype=float32)
i=0
pointed_idx Tensor("Decoder_6/Reshape:0", shape=(?, 30, 1), dtype=float32)
next_inp Tensor("Decoder_6/Sum:0", shape=(?, 300), dtype=float32)
attention_states Tensor("Decoder_6/conv1d/Squeeze:0", shape=(?, 30, 50), dtype=float32)
output Tensor("Decoder_6/Decoder/gru_cell/add:0", shape=(?, 50), dtype=float32)
output Tensor("Decoder_6/Softmax:0", shape=(?, 30), dtype=float32)
i=1
pointed_idx Tensor("Decoder_6/Reshape_2:0", shape=(?, 30, 1), dtype=float32)
next_inp Tensor("Decoder_6/Sum_2:0", shape=(?, 300), dtype=float32)
attention_states Tensor("Decoder_6/conv1d/Squeeze:0", shape=(?, 30, 50), dtype=float32)
output Tensor("Decoder_6/Decoder/gru_cell/add

In [50]:
dataset_ph = tf.placeholder(tf.string, shape=(None,))
dataset = tf.data.Dataset.from_tensor_slices(dataset_ph)
dataset = dataset.map(lambda string: tf.string_split([string]).values)
dataset = dataset.map(lambda words: (vocab.tokens2ids(words), tf.size(words)))

#source_target_dataset = tf.data.Dataset.zip((source_dataset, target_dataset))

sess.run(dataset, feed_dict=)

In [5]:
import sys
sys.stdout = sys.stderr
from six.moves import xrange # pylint: disable=redefined-builtin

import tensorflow as tf
from tensorflow.python.framework import dtypes
from tensorflow.python.ops.rnn_cell_impl import _linear # tf1.4
encoder_inputs = model.e_inputs_emb
decoder_inputs =model.d_inputs
attention_states = model.attention_states
initial_state = model.e_state
cell = model.d_cell
feed_prev=True
dtype=dtypes.float32
scope=None
print encoder_inputs
print decoder_inputs


Tensor("embedding_lookup_1:0", shape=(4, 31, 300), dtype=float32)
Tensor("concat_2:0", shape=(4, 5), dtype=int32)


In [7]:

with tf.variable_scope(scope or "point_decoder", reuse=tf.AUTO_REUSE):
    print 'encoder_inputs',encoder_inputs
    print 'decoder_inputs', decoder_inputs
    print 'attention_states', attention_states
    attn_length = attention_states.get_shape()[1].value
    attn_size = attention_states.get_shape()[2].value
    states = [initial_state]
    outputs = []

    attnw = tf.get_variable("AttnW", [1, attn_size, attn_size])
    attention_states = tf.nn.conv1d(attention_states, attnw, 1, 'SAME')
    attnv = tf.get_variable("AttnV", [attn_size])

    def attention(output):
        print 'attention_states', attention_states
        print 'output', output
        y = _linear(output, attn_size, True)
        y = tf.reshape(y, [-1, 1, attn_size])
        attention_vectors = tf.nn.softmax(tf.reduce_sum(attnv * tf.tanh(y + attention_states), axis=2))
        return attention_vectors

    for i, d in enumerate(tf.unstack(decoder_inputs, axis=1)):
        print i
        if i > 0:
            tf.get_variable_scope().reuse_variables()
        pointed_idx = d
        # in testing, inputs to decoder won't be used except the first one.
        if feed_prev and i > 0:
            # take argmax, convert the pointed index into one-hot, and get the pointed encoder_inputs by multiplying and reduce_sum.
            pointed_idx =  tf.argmax(output, axis=1)
        
        pointed_idx = tf.reshape(tf.one_hot(pointed_idx, depth=attn_length), [-1, attn_length, 1]) 
        inp = tf.reduce_sum(encoder_inputs * pointed_idx, axis=1) 
        print 'pointed_idx',pointed_idx
        print 'next_inp', inp
        output, state = cell(inp, states[-1])
        output = attention(output)
        states.append(state)
        outputs.append(output)


encoder_inputs Tensor("embedding_lookup_1:0", shape=(4, 31, 300), dtype=float32)
decoder_inputs Tensor("concat_2:0", shape=(4, 5), dtype=int32)
attention_states Tensor("point_decoder/conv1d/Squeeze:0", shape=(4, 31, 50), dtype=float32)
0
pointed_idx Tensor("point_decoder_1/Reshape:0", shape=(4, 31, 1), dtype=float32)
next_inp Tensor("point_decoder_1/Sum:0", shape=(4, 300), dtype=float32)
attention_states Tensor("point_decoder_1/conv1d/Squeeze:0", shape=(4, 31, 50), dtype=float32)
output Tensor("point_decoder/point_decoder/gru_cell/add_5:0", shape=(4, 50), dtype=float32)
1
pointed_idx Tensor("point_decoder_1/Reshape_2:0", shape=(4, 31, 1), dtype=float32)
next_inp Tensor("point_decoder_1/Sum_2:0", shape=(4, 300), dtype=float32)
attention_states Tensor("point_decoder_1/conv1d/Squeeze:0", shape=(4, 31, 50), dtype=float32)
output Tensor("point_decoder/point_decoder/gru_cell/add_6:0", shape=(4, 50), dtype=float32)
2
pointed_idx Tensor("point_decoder_1/Reshape_4:0", shape=(4, 31, 1), dtype=fl

In [ ]:
dataset_type = getattr(datasets, config.dataset_type)
dataset = dataset_type(config.dataset_path, vocab)
sources, targets = dataset.symbolized
sources = sources[:3]
sources = [' '.join([str(x) for x in s]) for s in sources]

print sources
dataset = dataset.map(lambda string: tf.string_split([string]).values)
# sources = tf.keras.preprocessing.sequence.pad_sequences(sources, padding='post', truncating='post')
# print sources 
# sources = sources.map(lambda words, size: (table.lookup(words), size))
# print sources

#print tf.data.Dataset.from_tensor_slices(sources)
#print tf.data.Dataset(sources)
#print tf.keras.preprocessing.sequence.pad_sequences(dataset.symbolized)
#tf.data.Dataset.zip(dataset.symbolized)
# for i, (source, target) in enumerate(dataset.symbolized):
#     print source, target
#     print dataset.original_sources[i], [dataset.original_sources[i][t] for t in target]
#     if dataset.targets[i] != ['-'] and not ids:
#         print dataset.texts[i]
#         print dataset.values[i].split(':')[0]
#         print i, dataset.original_sources[i], dataset.targets[i], ids
    
    

In [ ]:
def tokens2ids(words):
    return [vocab.token2id(w) for w in words]

infer_batch = tf.placeholder(tf.string, shape=(None,))
dataset = tf.data.Dataset.from_tensor_slices(infer_batch)
dataset = dataset.map(lambda string: tf.string_split([string]).values)
dataset = dataset.map(lambda words: (words, tf.size(words)))
dataset = dataset.map(lambda words, size: (tokens2ids(words), size))
print dataset